In [3]:
import pandas as pd
import numpy as np

### Preserve The CASEIDs

In [5]:
drug_2010 = pd.read_stata('Data_File/2010.dta', convert_categoricals=False).set_index('CASEID')
drug_2011 = pd.read_stata('Data_File/2011.dta', convert_categoricals=False).set_index('CASEID')
drug_2012 = pd.read_stata('Data_File/2012.dta', convert_categoricals=False).set_index('CASEID')
drug_2013 = pd.read_stata('Data_File/2013.dta', convert_categoricals=False).set_index('CASEID')
drug_2014 = pd.read_stata('Data_File/2014.dta', convert_categoricals=False).set_index('CASEID')

In [6]:
#testing for 2012
drug_2012 = pd.read_stata('data/2012.dta', convert_categoricals=False).set_index('CASEID')
crime_2012 = drug_2012[["SNYSTOLE", "SNYSELL", "BOOKED", "LOCJAIL"]]
crime = crime_2012

In [3]:
# only contains the variables with <40% missingness
crime_2010 = drug_2010[["SNYSTOLE", "SNYSELL", "BOOKED", "LOCJAIL"]]
crime_2011 = drug_2011[["SNYSTOLE", "SNYSELL", "BOOKED", "LOCJAIL"]]
crime_2012 = drug_2012[["SNYSTOLE", "SNYSELL", "BOOKED", "LOCJAIL"]]
crime_2013 = drug_2013[["SNYSTOLE", "SNYSELL", "BOOKED", "LOCJAIL"]]
crime_2014 = drug_2014[["SNYSTOLE", "SNYSELL", "BOOKED", "LOCJAIL"]]

#combine the dataframes
crime = pd.concat([crime_2010, crime_2011, crime_2012, crime_2013, crime_2014])
crime

,SNYSTOLE,SNYSELL,BOOKED,LOCJAIL
CASEID,,,,
1,1,1,2,0
2,1,1,2,0
3,99,99,2,0
4,1,1,2,0
5,1,1,1,0
...,...,...,...,...
55267,1,1,2,0
55268,1,1,1,0
55269,1,1,2,0


### Clean the data (Non-Missing Data)

In [7]:
# Replace non-missing data with meaningful descriptions
replace = {
"SNYSTOLE_replace" : {1: "0", 2: "1-2", 3:"3-5", 4: "6-9", 5: ">=10"}, #85:bad data; 89: LEGITIMATE SKIP; 94:don't know; 97:refused; 98:blank; 99:skip
"SNYSELL_replace" : {1: "0", 2: "1-2", 3:"3-5", 4: "6-9", 5: ">=10"}, #85:bad data; 89: LEGITIMATE SKIP; 94:don't know; 97:refused; 98:blank; 99:skip
"BOOKED_replace" : {1: "Yes", 2: "No"}, #3: Yes LOGICALLY ASSIGNED; 85:bad data; 94:don't know; 97:refused; 98:blank;
"LOCJAIL_replace" : {0:"No/Unknown", 1: "Yes"}
}

for col in crime.columns:
    crime[col] = crime[col].replace(replace[col + "_replace"])
        

/tmp/ipykernel_294/1736908978.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime[col] = crime[col].replace(replace[col + "_replace"])


### Rename the columns of Crime Variables

In [8]:
# Rename the columns
crime = crime.rename({"SNYSTOLE":'Ever_Steal_50', "SNYSELL": 'Sell_Illgal_Drugs', 
               "BOOKED":"Ever_Booked_Arrested", "LOCJAIL":"Drug_or_Alc_Past_year"},axis=1)

In [9]:
crime

,Ever_Steal_50,Sell_Illgal_Drugs,Ever_Booked_Arrested,Drug_or_Alc_Past_year
CASEID,,,,
1,99,99,No,No/Unknown
2,0,0,No,No/Unknown
3,0,0,No,No/Unknown
4,0,0,No,No/Unknown
5,0,0,No,No/Unknown
...,...,...,...,...
55264,0,0,No,No/Unknown
55265,99,99,Yes,No/Unknown
55266,0,0,No,No/Unknown


### Impute Missing Data

In [12]:
for col in crime.columns:
    print(crime[col].value_counts())

0       37126
99      17399
1-2       424
3-5        99
>=10       92
97         56
6-9        46
94         22
98          4
Name: Ever_Steal_50, dtype: int64
0       36488
99      17399
1-2       508
>=10      445
3-5       216
6-9       103
97         79
94         26
98          4
Name: Sell_Illgal_Drugs, dtype: int64
No     46593
Yes     8011
3        514
97        97
94        49
98         3
85         1
Name: Ever_Booked_Arrested, dtype: int64
No/Unknown    55156
Yes             112
Name: Drug_or_Alc_Past_year, dtype: int64


There are **86854** missing/bad data in Ever_Steal_50, **87052** in Sell_Illgal_Drugs, **3269** in Ever_Booked_Arrested, **0** in Drug_or_Alc_Past_year

In [13]:
# short versions for the column names
steal = "Ever_Steal_50"
sell = "Sell_Illgal_Drugs"
booked = "Ever_Booked_Arrested"

#Sample values from the observed values to fill the missing values
steal_not_missing = crime[(crime[steal] != 85) & (crime[steal] != 94) & (crime[steal] != 97) 
                          & (crime[steal] != 98) & (crime[steal] != 99) & (crime[steal] != 89)]
sell_not_missing = crime[(crime[sell] != 85) & (crime[sell] != 94) & (crime[sell] != 97) & 
                         (crime[sell] != 98) & (crime[sell] != 99) & (crime[sell] != 89)]
booked_not_missing = crime[(crime[booked] != 85) & (crime[booked] != 94) & (crime[booked] != 97) & 
                           (crime[booked] != 98) & (crime[booked] != 3)]

fill_values_steal = np.random.choice(steal_not_missing[steal], crime.shape[0]-steal_not_missing.shape[0])
fill_values_sell = np.random.choice(steal_not_missing[sell], crime.shape[0]-sell_not_missing.shape[0])
fill_values_booked = np.random.choice(booked_not_missing[booked], crime.shape[0]-booked_not_missing.shape[0])

#fill the missing values
steal_missing = (crime[steal] == 85) | (crime[steal] == 94) | (crime[steal] == 97) | (crime[steal] == 98) | (crime[steal] == 99) | (crime[steal] == 89)
sell_missing = (crime[sell] == 85) | (crime[sell] == 94) | (crime[sell] == 97) | (crime[sell] == 98) | (crime[sell] == 99) | (crime[sell] == 89)
booked_missing = (crime[booked] == 85) | (crime[booked] == 94) | (crime[booked] == 97) | (crime[booked] == 98) | (crime[booked] == 3)


crime.loc[steal_missing, steal] = fill_values_steal
crime.loc[sell_missing, sell] = fill_values_sell
crime.loc[booked_missing, booked] = fill_values_booked

crime

,Ever_Steal_50,Sell_Illgal_Drugs,Ever_Booked_Arrested,Drug_or_Alc_Past_year
CASEID,,,,
1,1-2,0,No,No/Unknown
2,0,0,No,No/Unknown
3,0,0,No,No/Unknown
4,0,0,No,No/Unknown
5,0,0,No,No/Unknown
...,...,...,...,...
55264,0,0,No,No/Unknown
55265,0,0,Yes,No/Unknown
55266,0,0,No,No/Unknown


### Merge with other clean dataset with aggregated data from 2010 to 2014

Waiting for data...

In [10]:
#religion_crime = crime.merge(religion, left_on='CASEID', right_on='CASEID', how='inner')

In [11]:
#religion_crime.to_csv('Data_File/religion_crime.csv')

In [23]:
demo_religion_2012 = pd.read_csv('data/religion_demographics_2012.csv')

In [24]:
demo_religion_2012 = demo_religion_2012.drop(columns=['Unnamed: 0'])

In [25]:
demo_religion_2012.columns

Index(['CASEID', 'Age', 'Health', 'Sex', 'Marriage', 'Education', 'Race',
       'Employment', 'religion_services', 'religion_important',
       'religion_influence_decision', 'religion_share_important'],
      dtype='object')

In [29]:
demo_religion_2012.shape

(55268, 12)

In [26]:
demo_religion_crime_2012 = crime.merge(demo_religion_2012, left_on='CASEID', right_on='CASEID', how='inner')

In [28]:
demo_religion_crime_2012.columns

Index(['CASEID', 'Ever_Steal_50', 'Sell_Illgal_Drugs', 'Ever_Booked_Arrested',
       'Drug_or_Alc_Past_year', 'Age', 'Health', 'Sex', 'Marriage',
       'Education', 'Race', 'Employment', 'religion_services',
       'religion_important', 'religion_influence_decision',
       'religion_share_important'],
      dtype='object')

In [ ]:
demo_religion_crime_2012.

In [18]:
demo_religion_crime_2012.to_csv('data/demo_religion_crime_2012.csv')

In [20]:
demo_religion_crime_2012.columns

Index(['CASEID', 'Ever_Steal_50', 'Sell_Illgal_Drugs', 'Ever_Booked_Arrested',
       'Drug_or_Alc_Past_year', 'Unnamed: 0', 'Age', 'Health', 'Sex',
       'Marriage', 'Education', 'Race', 'Employment', 'religion_services',
       'religion_important', 'religion_influence_decision',
       'religion_share_important'],
      dtype='object')